<h1>NYC Rent Affordability Investigation:<br>Gentrification ML Analysis</h1>

This is an investigation of rent affordability in NYC using neighborhood-level and borough-level from 2012-2022. 

I am using median rent data from the [Streeteasy Dashboard](https://streeteasy.com/blog/data-dashboard/), and median income data from the [U.S. Census American Community Survey](https://www.census.gov/programs-surveys/acs/data.html).

Here, I'll be using sci-kit learn to conduct unsupervised learning to create clusters Brooklyn neighborhoods based on Gentrification and Affordability indicators. This will give us insight into the different stages of gentrification.

# Importing packages and modules

In [1]:
# loading in custom utils
from utils.psql_connection_tools import get_engine
from utils.gentrification_tools import (engineer_gentrification_features, analyze_temporal_patterns, 
                                        detect_anomalies, cluster_gentrification_stages)

In [4]:
# stats, plotting, and access
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# postgres connection and analyses
import json
from google.cloud import bigquery
from google.oauth2 import service_account
# estimators/models
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
# preprocessing & engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# model scoring
from sklearn.metrics import silhouette_score
# organization
import warnings
warnings.filterwarnings('ignore')

# Importing relevant data from a BigQuery Postgres Database

In [5]:
# configure Big Query connection
PROJECT_ID = "rent-affordability"
DATASET_ID = "nyc_analysis"
service_account_info = json.loads(os.environ["GOOGLE_CREDENTIALS_JSON"])
credentials = service_account.Credentials.from_service_account_info(service_account_info)
client = bigquery.Client(project=PROJECT_ID, credentials=credentials)

In [40]:
# importing median rent price and year-over-year changes in rent price data
query = """
SELECT
    mr.neighborhood,
    mr.borough,
    mr.year,
    AVG(mr.all_apartments) AS avg_median_all_apts,
    AVG(mr.one_bedroom) AS avg_median_1bdr_apts,
    AVG(mr.three_plus_bedroom) AS avg_median_3bdr_apts,
    AVG(yrc.yoy_change_pct_all) AS yoy_change_pct_all,
    AVG(yrc.yoy_change_pct_1bdr) AS yoy_change_pct_1bdr,
    AVG(yrc.yoy_change_pct_3bdr) AS yoy_change_pct_3bdr
FROM nyc_analysis.fact_median_rent mr
JOIN nyc_analysis.agg_yoy_rent_change yrc ON yrc.borough_name = mr.borough
    AND yrc.neighborhood_name = mr.neighborhood
    AND yrc.year = mr.year
WHERE mr.borough = 'Brooklyn' AND yrc.months_of_data = 12
GROUP BY neighborhood, borough, year
ORDER BY neighborhood, borough, year
"""

rent_df = client.query(query).to_dataframe()
rent_df

,neighborhood,borough,year,avg_median_all_apts,avg_median_1bdr_apts,avg_median_3bdr_apts,yoy_change_pct_all,yoy_change_pct_1bdr,yoy_change_pct_3bdr
0,Bath Beach,Brooklyn,2015,1815.500000000,0E-9,712.500000000,5.630000000,None,None
1,Bath Beach,Brooklyn,2016,2041.083333333,0E-9,2341.666666667,12.430000000,None,228.650000000
2,Bath Beach,Brooklyn,2017,1943.416666667,360.416666667,1772.583333333,-4.790000000,None,-24.300000000
3,Bath Beach,Brooklyn,2018,1824.833333333,1154.083333333,1953.916666667,-6.100000000,220.210000000,10.230000000
4,Bath Beach,Brooklyn,2019,1743.250000000,1614.000000000,1015.833333333,-4.470000000,39.850000000,-48.010000000
...,...,...,...,...,...,...,...,...,...
535,Windsor Terrace,Brooklyn,2020,2670.166666667,2312.416666667,2194.583333333,0E-9,-0.200000000,-23.540000000
536,Windsor Terrace,Brooklyn,2021,2634.000000000,2207.666666667,2989.583333333,-1.350000000,-4.530000000,36.230000000
537,Windsor Terrace,Brooklyn,2022,3165.500000000,2680.583333333,1378.916666667,20.180000000,21.420000000,-53.880000000
538,Windsor Terrace,Brooklyn,2023,3221.500000000,2998.166666667,904.166666667,1.770000000,11.850000000,-34.430000000


In [41]:
# importing median HH income and year-over-year changes in HH income data
query = """
    SELECT 
        n.name AS neighborhood,
        yic.borough_name AS borough,
        yic.year AS year,
        yic.yoy_change_pct_all_hhs,
        yic.yoy_change_pct_singles,
        yic.yoy_change_pct_other_kids,
        yic.yoy_change_pct_married_kids,
        mi.income_all_hhs AS median_all_hhs,
        mi.income_singles AS median_singles,
        mi.income_married_kids AS median_married_kids,
        mi.income_other_kids AS median_other_kids
    FROM nyc_analysis.agg_yoy_income_change yic
    JOIN nyc_analysis.ref_neighborhoods n ON n.name = yic.neighborhood_name
    JOIN nyc_analysis.ref_district_neighborhoods dn ON dn.neighborhood_id = n.neighborhood_id 
    JOIN nyc_analysis.fact_median_income mi ON mi.district_id = dn.district_id
        AND mi.year = yic.year 
    WHERE yic.borough_name = 'Brooklyn'
    ORDER BY neighborhood, year
    """
income_df = client.query(query).to_dataframe()
income_df

,neighborhood,borough,year,yoy_change_pct_all_hhs,yoy_change_pct_singles,yoy_change_pct_other_kids,yoy_change_pct_married_kids,median_all_hhs,median_singles,median_married_kids,median_other_kids
0,Bath Beach,Brooklyn,2012,None,None,None,None,50019.000000000,28739.000000000,33928.000000000,440.000000000
1,Bath Beach,Brooklyn,2013,-5.490000000,17.280000000,7.270000000,0.190000000,47272.000000000,33705.000000000,33992.000000000,472.000000000
2,Bath Beach,Brooklyn,2014,1.010000000,-12.050000000,-27.750000000,-2.430000000,47748.000000000,29645.000000000,33165.000000000,341.000000000
3,Bath Beach,Brooklyn,2015,4.780000000,-11.780000000,18.480000000,-3.560000000,50029.000000000,26152.000000000,31985.000000000,404.000000000
4,Bath Beach,Brooklyn,2016,2.110000000,39.140000000,-41.090000000,4.040000000,51083.000000000,36387.000000000,33277.000000000,238.000000000
...,...,...,...,...,...,...,...,...,...,...,...
355,Windsor Terrace,Brooklyn,2017,6.610000000,-10.560000000,None,None,56787.000000000,41720.000000000,0E-9,0E-9
356,Windsor Terrace,Brooklyn,2018,9.990000000,20.360000000,None,None,62458.000000000,50214.000000000,0E-9,0E-9
357,Windsor Terrace,Brooklyn,2019,8.670000000,-8.860000000,None,None,67871.000000000,45767.000000000,0E-9,0E-9
358,Windsor Terrace,Brooklyn,2021,-13.300000000,16.910000000,None,None,58841.000000000,53504.000000000,0E-9,0E-9


In [42]:
# handling negative values = NAN
income_df['median_all_hhs'][income_df['median_all_hhs'] < 0] = np.nan
income_df['median_singles'][income_df['median_singles'] < 0] = np.nan
income_df['median_other_kids'][income_df['median_other_kids'] < 0] = np.nan
income_df['median_married_kids'][income_df['median_married_kids'] < 0] = np.nan

# merging the income and rent datasets, to create one primate dataframe
rent_income_change_df = pd.merge(income_df, rent_df, how='inner', 
                                 on=['neighborhood','borough','year'])
# removing columns that are no longer needed
rent_income_change_df = rent_income_change_df.drop(['borough'], axis=1)
# reconciling duplicate neighborhood+year entries, due to PUMA district names (e.g. Crown Heights North/South)
rent_income_change_df = rent_income_change_df.groupby(['neighborhood', 'year']).mean().reset_index()
rent_income_change_df

,neighborhood,year,yoy_change_pct_all_hhs,yoy_change_pct_singles,yoy_change_pct_other_kids,yoy_change_pct_married_kids,median_all_hhs,median_singles,median_married_kids,median_other_kids,avg_median_all_apts,avg_median_1bdr_apts,avg_median_3bdr_apts,yoy_change_pct_all,yoy_change_pct_1bdr,yoy_change_pct_3bdr
0,Bath Beach,2015,4.78,-11.78,18.48,-3.56,50029.0,26152.0,31985.0,404.0,1815.5,0.0,712.5,5.63,NaN,NaN
1,Bath Beach,2016,2.11,39.14,-41.09,4.04,51083.0,36387.0,33277.0,238.0,2041.083333,0.0,2341.666667,12.43,NaN,228.65
2,Bath Beach,2017,4.72,-22.73,-53.78,7.7,53493.0,28116.0,35838.0,110.0,1943.416667,360.416667,1772.583333,-4.79,NaN,-24.3
3,Bath Beach,2018,1.8,-4.09,156.36,-5.43,54458.0,26967.0,33893.0,282.0,1824.833333,1154.083333,1953.916667,-6.1,220.21,10.23
4,Bath Beach,2019,4.92,38.65,27.66,-0.77,57139.0,37391.0,33631.0,360.0,1743.25,1614.0,1015.833333,-4.47,39.85,-48.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Windsor Terrace,2017,6.61,-10.56,NaN,NaN,56787.0,41720.0,0.0,0.0,2515.333333,2123.166667,2756.583333,-0.08,-0.32,-15.92
276,Windsor Terrace,2018,9.99,20.36,NaN,NaN,62458.0,50214.0,0.0,0.0,2550.083333,2200.916667,3300.333333,1.38,3.66,19.73
277,Windsor Terrace,2019,8.67,-8.86,NaN,NaN,67871.0,45767.0,0.0,0.0,2670.25,2317.166667,2870.166667,4.71,5.28,-13.03
278,Windsor Terrace,2021,-13.3,16.91,NaN,NaN,58841.0,53504.0,0.0,0.0,2634.0,2207.666667,2989.583333,-1.35,-4.53,36.23


In [44]:
rent_income_change_df['neighborhood'].unique()

array(['Bath Beach', 'Bay Ridge', 'Bedford-Stuyvesant', 'Bensonhurst',
       'Borough Park', 'Brighton Beach', 'Brooklyn Heights',
       'Brownsville', 'Bushwick', 'Canarsie', 'Carroll Gardens',
       'Coney Island', 'Crown Heights', 'Downtown Brooklyn',
       'Dyker Heights', 'East New York', 'Flatbush', 'Fort Greene',
       'Gravesend', 'Greenpoint', 'Kensington', 'Midwood', 'Park Slope',
       'Prospect Heights', 'Prospect Lefferts Gardens', 'Red Hook',
       'Sheepshead Bay', 'Sunset Park', 'Williamsburg', 'Windsor Terrace'],
      dtype=object)

# Machine Learning Gentrification Analysis

## Creating Features for the Gentrification Analysis

Creating a dataframe that contains data analyzing year-over-year changes in rent and income percentage values, affordability ratios, volatility, rent price acceleration, gentrification intensity, and other variables at the population-group and apartment-type level to develop neighborhood-level statistical information to supplement our machine learning analysis.

In [45]:
# Feature Engineering for Gentrification Detection
df_temp1 = engineer_gentrification_features(rent_income_change_df) 
print(df_temp1.columns)
df_temp1.head()

ZeroDivisionError: float division by zero

Creating a dataframe that documents neighborhood-level frequency of abnormally high Year-over-Year increases in rent price (e.g. 8%, 12%, 15%, or higher.)

In [6]:
# Time Series Pattern Analysis
df_temp2 = analyze_temporal_patterns(rent_income_change_df)
df_temp2.head()

,neighborhood,all_consec_8pct,all_consec_12pct,all_consec_15pct,1bdr_consec_8pct,1bdr_consec_12pct,1bdr_consec_15pct,3bdr_consec_8pct,3bdr_consec_12pct,3bdr_consec_15pct
0,Bath Beach,1,1,0,0,0,0,1,0,0
1,Bay Ridge,0,0,0,0,0,0,0,0,0
2,Bedford-Stuyvesant,1,0,0,0,0,0,0,0,0
3,Bensonhurst,2,1,1,1,0,0,1,0,0
4,Bushwick,2,2,1,1,0,0,2,0,0


## Using Unsupervised Machine Learning to Cluster Brooklyn Neighborhoods

In [34]:
# Clustering to identify stages of gentrification
cluster_features = ['all_yoy_rent_change_mean', 'all_yoy_rent_change_std', 'all_years_high_increase', 
                    'all_price_acceleration', 'all_total_rent_change', 'all_rent_volatility', 
                    'all_total_income_change', 'all_income_volatility', 
                    'all_consec_8pct', 'all_consec_12pct', 'all_consec_15pct']
df_temp3, kmeans, fitted_scalar = cluster_gentrification_stages(df_temp1.merge(df_temp2, how='inner',
                                                                                on='neighborhood'),
                                                                cluster_features)
# viewing clustering results
df_temp3[['neighborhood','gentrification_stage']].sort_values('gentrification_stage')

Optimal number of clusters: 5


,neighborhood,gentrification_stage
6,East Flatbush,0
11,Sheepshead Bay,0
10,Midwood,0
8,Flatbush,0
12,Sunset Park,1
1,Bay Ridge,1
7,East New York,2
2,Bedford-Stuyvesant,2
4,Bushwick,3
5,Canarsie,4


In [ ]:
# 6. Interpreting Clusters and Labelling Gentrification Stages
def interpret_clusters(features_df):
    """
    Analyze cluster characteristics to label gentrification stages
    """
    cluster_summary = features_df.groupby('gentrification_stage').agg(
        {x: 'mean' for x in cluster_features}).round(2)
    
    cluster_summary.columns = [
        'avg_rent_change', 'avg_years_high_increase', 'avg_intensity',
        'avg_acceleration', 'avg_total_change', 'avg_consecutive', 'count'
    ]
    
    print("Cluster Characteristics:")
    print(cluster_summary)
    
    # Manual labeling based on characteristics (adjust based on your results)
    stage_labels = {
        0: "Stable/Pre-Gentrification",
        1: "Early Gentrification", 
        2: "Active Gentrification",
        3: "Advanced Gentrification",
        4: "Post-Gentrification/Saturated"
    }
    
    features_df['stage_label'] = features_df['gentrification_stage'].map(
        lambda x: stage_labels.get(x, f"Stage_{x}")
    )
    
    return features_df, cluster_summary

In [ ]:
# 7. Visualizing results of K-Means Cluster Analysis
def create_visualizations(features_df):
    """
    Create visualizations to understand the results
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Scatter plot: Rent change vs Gentrification intensity
    axes[0,0].scatter(
        features_df['rent_change_mean'], 
        features_df['gentrification_intensity'],
        c=features_df['gentrification_stage'], 
        cmap='viridis', 
        alpha=0.7
    )
    axes[0,0].set_xlabel('Average YoY Rent Change %')
    axes[0,0].set_ylabel('Gentrification Intensity')
    axes[0,0].set_title('Neighborhoods by Gentrification Stage')
    
    # Box plot of rent changes by cluster
    features_df.boxplot(
        column='rent_change_mean', 
        by='stage_label', 
        ax=axes[0,1]
    )
    axes[0,1].set_title('Rent Change Distribution by Stage')
    
    # Anomaly detection visualization
    colors = ['red' if x else 'blue' for x in features_df['is_anomaly']]
    axes[1,0].scatter(
        features_df['gentrification_intensity'],
        features_df['anomaly_score'],
        c=colors,
        alpha=0.7
    )
    axes[1,0].set_xlabel('Gentrification Intensity')
    axes[1,0].set_ylabel('Anomaly Score')
    axes[1,0].set_title('Anomaly Detection (Red = Anomalies)')
    
    # Years of high increases
    features_df.boxplot(
        column='years_high_increase',
        by='stage_label',
        ax=axes[1,1]
    )
    axes[1,1].set_title('Years of High Increases by Stage')
    
    plt.tight_layout()
    plt.show()

Using the Isolation Forest ML technique to identify neighborhoods with anomalous patterns in rent/income volatility, rent price acceleration, total changes in Household income, and more:

In [7]:
# Detecting anomalies across All Apartment and All Household Data 
anomaly_features = ['all_yoy_rent_change_mean', 'all_yoy_rent_change_std', 'all_years_high_increase', 
                             'all_price_acceleration', 'all_total_rent_change', 'all_rent_volatility', 
                             'all_total_income_change', 'all_income_volatility']
df_temp3 = detect_anomalies(df_temp1, anomaly_features, 
                            'all_apts_anomaly_score', 'all_apts_is_anomaly')
df_temp3.head()

,neighborhood,all_yoy_rent_change_mean,all_yoy_rent_change_std,all_yoy_rent_change_max,all_yoy_rent_change_min,all_years_high_increase,all_years_extreme_increase,all_yoy_income_change_mean,all_yoy_income_change_std,all_yoy_income_change_max,...,other_kids_rent_income_ratio,married_kids_rent_income_ratio,all_gentrification_intensity,1bdr_gentrification_intensity,3bdr_gentrification_intensity,all_price_acceleration,1bdr_price_acceleration,3bdr_price_acceleration,all_apts_anomaly_score,all_apts_is_anomaly
0,Bath Beach,0.255714,7.374137,12.43,-6.10,1,0,3.154286,2.735811,5.78,...,120.762287,0.835607,0.167600,0.000000,0.0,12.174286,3.290000,7.754000,0.025089,False
1,Bay Ridge,3.070000,2.701148,4.98,1.16,0,0,11.860000,NaN,11.86,...,181.547234,1.334071,0.000000,0.000000,0.0,1.910000,2.595000,NaN,0.002876,False
2,Bedford-Stuyvesant,7.293333,2.778207,9.87,4.35,0,0,2.520000,16.277598,14.03,...,65.849642,2.500775,0.000000,0.000000,0.0,2.576667,1.273333,1.373333,0.042079,False
3,Bensonhurst,3.735556,8.729910,20.34,-6.62,2,1,1.955556,3.729896,5.78,...,104.771786,0.845223,0.318274,0.000000,0.0,16.604444,6.272500,6.320000,-0.014744,True
4,Bushwick,15.000000,3.889087,17.75,12.25,2,1,23.770000,NaN,23.77,...,154.358496,2.451500,1.177481,0.555858,0.0,2.750000,10.050000,0.315000,-0.001233,True


In [ ]:
# Step 8: Main execution function
def main():
    """
    Execute the complete gentrification analysis pipeline
    """
    print("Step 1: Loading data...")
    # df = load_and_prepare_data()  # Uncomment when you have DB connection
    
    # For demonstration, create sample data
    np.random.seed(42)
    neighborhoods = ['Park Slope', 'Williamsburg', 'Crown Heights', 'Bushwick', 
                    'Red Hook', 'DUMBO', 'Bedford-Stuyvesant', 'Greenpoint']
    
    # Create sample data - replace with your actual data loading
    sample_data = []
    for neighborhood in neighborhoods:
        for year in range(2015, 2024):
            sample_data.append({
                'neighborhood_name': neighborhood,
                'borough_name': 'Brooklyn',
                'year': year,
                'yoy_rent_change_pct': np.random.normal(5 + hash(neighborhood) % 10, 3),
                'yoy_income_change_pct': np.random.normal(2, 2),
                'median_rent': 2000 + year * 100 + hash(neighborhood) % 500,
                'median_income': 50000 + year * 1000 + hash(neighborhood) % 10000
            })
    
    df = pd.DataFrame(sample_data)
    
    print("Step 2: Engineering features...")
    features_df = engineer_gentrification_features(df)
    
    print("Step 3: Analyzing temporal patterns...")
    temporal_features = analyze_temporal_patterns(df)
    features_df = features_df.merge(temporal_features, on='neighborhood_name')
    
    print("Step 4: Detecting anomalies...")
    features_df = detect_anomalies(features_df)
    
    print("Step 5: Clustering neighborhoods...")
    features_df, model, scaler = cluster_gentrification_stages(features_df)
    
    print("Step 6: Interpreting clusters...")
    features_df, cluster_summary = interpret_clusters(features_df)
    
    print("Step 7: Creating visualizations...")
    create_visualizations(features_df)
    
    print("\nResults Summary:")
    print(features_df[['neighborhood_name', 'stage_label', 'is_anomaly', 
                      'gentrification_intensity', 'anomaly_score']].sort_values('gentrification_intensity', ascending=False))
    
    return features_df, model, scaler

if __name__ == "__main__":
    results, model, scaler = main()